In [64]:
import pandas as pd
import numpy as np

df_train = pd.read_csv('data/train_solved.csv')
df_test = pd.read_csv('data/test_solved.csv')
print(df_train.shape)
print(df_test.shape)
df_test.head()

(1100, 52)
(350, 51)


,Age,DistanceFromHome,Education,EnvironmentSatisfaction,JobInvolvement,JobSatisfaction,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,PerformanceRating,...,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,OverTime_No,OverTime_Yes
0,40,9,4,3,3,3,3975,3,11,3,...,0,0,0,0,0,1,0,0,1,0
1,53,7,2,4,3,3,18606,3,18,3,...,0,0,0,0,0,1,0,0,1,0
2,42,2,4,1,2,4,6781,3,23,4,...,0,0,0,0,0,0,0,1,1,0
3,34,11,3,3,2,2,4490,4,11,3,...,0,0,0,0,0,0,1,0,1,0
4,32,1,1,4,3,1,2956,1,13,3,...,0,0,1,0,0,0,0,1,1,0


In [58]:
# 标准化
from sklearn.preprocessing import MinMaxScaler
std = MinMaxScaler()
df_train.iloc[:,1:] = std.fit_transform(df_train.iloc[:,1:])
df_test = std.transform(df_test)

## 建模

In [65]:
# 多模型交叉验证
import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
models ={
    'LR':LogisticRegression(solver='liblinear'),
    'SVM':SVC(),
    'RF':RandomForestClassifier(n_estimators=150),
}
for k,clf in models.items():
    print('the model is %s'%k)
    scores = cross_val_score(clf,train.iloc[:,1:],train['Attrition'])
    print('Mean accuracy is {}'.format(np.mean(scores)))
    print('-'*20)

the model is LR
Mean accuracy is 0.8772570681872178
--------------------
the model is SVM
Mean accuracy is 0.8381840500514771
--------------------
the model is RF
Mean accuracy is 0.8681990971727251
--------------------


In [66]:
# 数据分割 加网格搜索
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
x_train,x_test,y_train,y_test = train_test_split(
    df_train.iloc[:,1:],df_train['Attrition'],random_state=22)
params={
    'penalty':['l1', 'l2'],
    'C':np.arange(1,4.1,0.2),
}
estimator = LogisticRegression(solver='liblinear')
grid = GridSearchCV(estimator,param_grid=params,cv=10)
grid.fit(x_train,y_train)
print('最好的参数',grid.best_params_)
score = grid.score(x_test,y_test)
print('在测试集的得分',score)

最好的参数 {'C': 2.8, 'penalty': 'l2'}
在测试集的得分 0.8981818181818182


## 进行预测

In [45]:
estimator = LogisticRegression(solver='liblinear',C=1.9,penalty='l1')
estimator.fit(df_train.iloc[:,1:],df_train['Attrition'])
pred = estimator.predict(df_test)
pred = pd.DataFrame(pred)
pred.to_csv('data/2-pred.csv',index=False,)
print('Good predict')

Good predict


In [48]:
pred.shape

(350, 1)